## An example of how to construct a scheme to solve a PDE

### Part 1

Write an (explicit) FTCS finite difference scheme to solve the partial differential equation

$$\frac{\partial u}{\partial t}=\epsilon\frac{\partial u}{\partial x}+\kappa\frac{\partial^2u}{\partial x^2},\qquad 0\le x\le 1,\quad t>0,\quad \kappa>0,$$

with initial conditions $u(x,0)=f(x)$ and boundary conditions $u(0,t)=g_0(t)$ and $u(1,t)=g_1(t)$, where $f(x),g_0(t),g_1(t)$ are given functions.

### Answer

As usual, use $u_j^n$ to denote $u(x_j,t^n)$. Since it is _forward_ time and _centred_ space, write:
	\begin{align}
		\label{ut}\frac{\partial u}{\partial t}&=\frac{u_j^{n+1}-u_j^n}{\delta}+\mathcal{O}(\delta),\\
		\label{ux}\frac{\partial u}{\partial x}&=\frac{u_{j+1}^n-u_{j-1}^n}{2h}+\mathcal{O}(h^2),\\
		\label{uxx}\frac{\partial^2u}{\partial x^2}&=\frac{u_{j+1}^n+u_{j-1}^n-2u_j^n}{h^2}+\mathcal{O}(h^2).
	\end{align}
	Rearranging terms, we obtain the following scheme:
	\begin{equation}
		u_j^{n+1}=\biggl(\frac{\delta\epsilon}{2h}+\frac{\delta\kappa}{h^2}\biggr)u_{j+1}^n+\biggl(1-\frac{2\delta\kappa}{h^2}\biggr)u_j^n+\biggl(\frac{\delta\kappa}{h^2}-\frac{\delta\epsilon}{2h}\biggr)u_{j-1}^n,
	\end{equation}
	where $1\le j\le N$, $N=1/h$ (so that $x_0=0$ and $x_{N+1}=1$), $t^n=\delta n$, the initial conditions are $u_j^0=f(x_j)$ for $0\le j\le N+1$, and the boundary conditions are $u_0^n=g_0(t^n)$ and $u_{N+1}^n=g_1(t^n)$.

### Part 2

Implement your FTCS method above. Use the explicit initial-boundary conditions

$$f(x)=x(1-x),\qquad g_0(t)=0,\qquad g_1(t)=0.$$

To be precise, write a Python function that takes as input  
* the number of timesteps ``N`` the algorithm will take,  
* the space discretization step ``h``,  
* the time discretization step ``delta``,  
* the diffusion parameter ``kappa``,  
* the advection parameter ``epsilon``,    
and returns the coordinates ``x`` of the spatial grid, and the solution ``u`` at all timesteps computed. Additional helper functions may be implemented if helpful. All functions should be clearly documented.

### Answer

In [ ]:
import numpy

def FTCS(N, h, delta, kappa, epsilon):
    """
    Implement the FTCS algorithm for the equation
    u_t=epsilon*u_x+kappa*u_xx for x in [0,1]
    with initial condition u(x,0)=x(1-x)
    and boundary conditions u(0,t)=u(1,t)=0
    N = number of timesteps, h = space discretization step
    delta = time discretization step, kappa must be >0
    """
    #Get the x coordinates for the given discretization step
    x = numpy.linspace(0.0, 1.0, 1.0/h+1)
    #Initialize the mesh
    u = numpy.zeros((N+1,len(x)))
    #Initial condition
    u[0, :] = x*(1-x)
    for n in range(N):
        #boundary conditions
        u[n+1, 0] = 0
        u[n+1, -1] = 0
        for i in range(1, len(x)-1):
            #for points not on the boundary, use the FTCS scheme
            u[n+1, i] = u[n, i+1] * ((delta * kappa) / (h**2) + (delta * epsilon) / (2 * h)) + u[n, i] * (1 - (2 * delta * kappa / (h**2))) + u[n, i-1] * ((delta * kappa) / (h**2) - (delta * epsilon) / (2 * h))
    return x, u

### Part 3

Fix $\kappa=\epsilon h/2$.

Plot your results:  
* at the 100th timestep when $h=10^{-2}, \delta=10^{-3},\epsilon=2,\kappa=10^{-2}$;  
* at the 10th timestep when you set $\delta=h=10^{-2}$ but keep the other parameters the same.  

How would you interpret your results?

### Answer

In [ ]:
from matplotlib import pyplot

coord1,values1 = FTCS(100,0.01,0.001,0.01,2)
coord2,values2 = FTCS(10,0.01,0.01,0.01,2)

pyplot.plot(coord1,values1[100],'r-')
pyplot.xlabel('$x$')
pyplot.ylabel('$u(x,100)$')
pyplot.show()

pyplot.plot(coord2,values2[10],'b-')
pyplot.xlabel('$x$')
pyplot.ylabel('$u(x,10)$')
pyplot.show()

Notice that the scheme we are using in the second case is

$$u_j^{n+1}=2u_{j+1}^n-u_j^n,$$

in which the factor 2 gives the exponential behaviour we observe.

#### Remark

The last question has a more detailed explanation that I cannot cover in detail as we did not go into some technical aspects of schemes.

For those who are interested and want to study a bit more this topic, the idea is that the algorithm is consistent therefore if it is convergent then it is also stable (Lax's theorem). By using the condition $\kappa=\epsilon h/2$, the scheme becomes

$$u_j^{n+1}=ru_{j+1}^n+(1-r)u_j^n,$$

where $r=\delta\epsilon/h$. One can show that this scheme is convergent (hence stable) only when $r<1$, i.e. $\delta<h/\epsilon$.

In the previous question, in the first case the condition is satisfied, while in the second case it is not, hence the different behaviour.